In [82]:
import requests
import json
from sklearn.manifold import TSNE

import chardet
import re
import pandas as pd
import numpy as np
from pprint import pprint
import requests
from bixin import predict
import snownlp
import dm_pb2 as Danmaku
import sklearn

In [69]:
# 1.根据bvid请求得到cid
bvid = "BV1bc411H7Gv"
def get_cid():
    url = 'https://api.bilibili.com/x/player/pagelist?bvid=BV1bc411H7Gv'
    res = requests.get(url).text
    json_dict = json.loads(res)
    pprint(json_dict)
    return (json_dict["data"][0]["duration"], json_dict["data"][0]["cid"])
(max_time, cid) = get_cid()
max_time = max_time * 1000

print(cid)



{'code': 0,
 'data': [{'cid': 1105215233,
           'dimension': {'height': 1920, 'rotate': 0, 'width': 3840},
           'duration': 87,
           'first_frame': 'http://i0.hdslb.com/bfs/storyff/n230422a219gylliozq0r520l6kdz8f1_firsti.jpg',
           'from': 'vupload',
           'page': 1,
           'part': '【4K修复+60帧】小鬼 泰裤辣（原版）',
           'vid': '',
           'weblink': ''}],
 'message': '0',
 'ttl': 1}
1105215233


In [70]:
# 更新：新版api需要使用proto获取弹幕文件
def get_prot_dm(cid):
    url = 'https://api.bilibili.com/x/v2/dm/web/seg.so'
    params = {
        'type': 1,         # 弹幕类型
        'oid': cid,    # cid
        'segment_index': 1 # 弹幕分段
    }
    resp = requests.get(url, params)
    data = resp.content

    danmaku_seg = Danmaku.DmSegMobileReply()
    danmaku_seg.ParseFromString(data)

    return danmaku_seg.elems

danmu_proto = get_prot_dm(cid)

In [71]:
#得到弹幕 array
def to_list(prot):
    l = []
    for i in range(len(prot)):
        l.append({})
        l[i]["id"] = prot[i].id
        l[i]["progress"] = prot[i].progress
        l[i]["content"] = prot[i].content

    return l
listed_danmu = to_list(danmu_proto)
df = pd.DataFrame(listed_danmu)
mask = df['content'].str.len() > 10
df = df[~mask]
df['content'] = df['content'].str.replace('[^\w\s]', '')
mask = df['content'].str.len() < 1
df = df[~mask]
df.dropna(subset=['content'], inplace=True)
mask = df['content'].str.len() == 0
df = df[~mask]
print(df)

                       id  progress    content
0     1301402202560179456     23493         哥哥
1     1301445531851347456       324       我40帧
4     1301467215505580032     52312       油光满面
5     1301468690323432448     76163       音乐骤停
8     1301487277205294592     51629       脚趾扣地
...                   ...       ...        ...
3382  1310799634725521408     64658       要来力喜
3383  1310810228899852544      1924       我擦深圳
3384  1310812124808779264       659      我死在深圳
3386  1310828580598763776     30641        yue
3387  1310839748100811008      7618  什么都修复哈哈哈哈

[2609 rows x 3 columns]


In [72]:
#divide into 6 parts according to time variation
def classify_sentiment(s):
    if(s<0.166):
        return 0
    if(s<0.333):
        return 1
    if(s<0.5):
        return 2
    if(s<0.666):
        return 3
    if(s<0.833):
        return 4
    if(s<=1):
        return 5
def get_sentiment(text):
    s = snownlp.SnowNLP(text)
    return classify_sentiment(s.sentiments)
index = 0
for i in range(6):
    start_time = max_time * i / 6
    end_time = max_time * (i+1) / 6
    tmp = df[(df['progress'] >= start_time) & (df['progress'] <= end_time)]
    tmp = tmp['content'].value_counts()
    tmp = pd.DataFrame({'content': tmp.index, 'count': tmp.values})
    mask = tmp['count'].values <= 3
    tmp = tmp[~mask]
    print(tmp)
    tmp['emo'] = tmp['content'].apply(get_sentiment)
    tmp.to_csv("video_"+str(index)+".csv")
    index+=1

df.to_csv("video.csv")
counts = df['content'].value_counts()


       content  count
0         我测深圳     29
1       给我点鸡叫声     13
2   什么都修复只会害了你     11
3    什么都修只会害了你      9
4         我是丁真      9
5        我死在深圳      9
6         我40帧      7
7          泰裤辣      6
8       给我点滋尿声      5
9         我是深圳      4
10         抬裤拉      4
11        油光满面      4
     content  count
0        董明珠     12
1         病句     10
2       油光满面      9
3   如约而至地出现了      9
4        泰裤辣      9
5          油      6
6        泰裤啦      6
7        泰油辣      4
8       杰夫哈迪      4
9         好油      4
10    我还是在这里      4
  content  count
0     住医院     15
1    家电下乡     11
2     泰裤辣      7
3     yue      4
4     做阴乐      4
    content  count
0     那可太好了     25
1       泰裤辣     10
2   哈哈哈哈哈哈哈      9
3    真的吗我不信      8
4       真的吗      8
5      油光满面      6
6         油      6
7       我不信      6
8     那可泰好了      6
9        假的      5
10      好尴尬      5
11      尬死了      4
  content  count
0     要来力     56
1    开始吟唱     15
2    要来力喜      9
3     泰裤辣      7
4    全文背诵      5
5     我不管      4
  cont

In [73]:
# 2.根据cid请求弹幕，解析弹幕得到最终的数据

# def get_data(cid):
#     final_url = "https://api.bilibili.com/x/v1/dm/list.so?oid=" + str(cid)
#     final_res = requests.get(final_url)
#     final_res.encoding = chardet.detect(final_res.content)['encoding']
#     final_res = final_res.text
#     pattern = re.compile('<d.*?>(.*?)</d>')
#     match = re.compile(r'<d\s+[^>]*\bp="([^"]*)"')
#     pa = match.findall(final_res)
#
#     data = pattern.findall(final_res)
#     danmu_time = [float(item.split(',')[0]) for item in pa]
#     danmu_mode = [float(item.split(',')[1]) for item in pa]
#     danmu_size = [float(item.split(',')[2]) for item in pa]
#     danmu_color = [float(item.split(',')[3]) for item in pa]
#     danmu_abstime = [float(item.split(',')[4]) for item in pa]
#     danmu_pool = [float(item.split(',')[5]) for item in pa]
#     danmu_id = [(item.split(',')[6]) for item in pa]
#     danmu_rowid = [float(item.split(',')[7]) for item in pa]
#     print(len(data))
#     return (data, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid)
#
#
# (danmu_text, danmu_time, danmu_mode, danmu_size, danmu_color, danmu_abstime, danmu_pool, danmu_id, danmu_rowid) = get_data(cid)

In [74]:
# # 分割弹幕为八个部分
# # 视频持续时间
# max_time = max(danmu_time)
# divided_danmu = []
# for i in range(8):
#     time_start = max_time * (i/8)
#     time_stop = max_time * ((i+1)/8)
#     for j in range(len(danmu_time)):
#         if time_start <= danmu_time[j] <= time_stop:
#             divided_danmu.append([i, danmu_text[j]])
# print(len(danmu_time))

In [75]:
# export csv
# arr = np.array(divided_danmu)
# df = pd.DataFrame(arr)
# df.to_csv("video.csv", header=["label", "text"])

In [76]:
def get_relate(bvid):
    # 设置请求参数
    params = {
        "bvid": bvid,  # 你想获取推荐视频的原始视频的bid
        "num": 1,  # 获取的推荐视频数量，最多为40
    }

    # 发送请求
    response = requests.get("https://api.bilibili.com/x/web-interface/archive/related", params=params)

    # 解析响应
    if response.status_code == 200:
        data = response.json()
        recommend_list = data.get("data")
        # 处理推荐视频列表数据
        return recommend_list
    else:
        print("请求失败")

relate_list = get_relate(bvid)
print(relate_list[0])

{'aid': 442960465, 'videos': 1, 'tid': 126, 'tname': '人力VOCALOID', 'copyright': 1, 'pic': 'http://i0.hdslb.com/bfs/archive/4508d4fec623d274cc1f475613a980e96eea5e40.jpg', 'title': '⚡泰 裤 辣—最阴间版本⚡', 'pubdate': 1682330406, 'ctime': 1682330406, 'desc': '如果，你们看了视频，能给up主两个小小的，圆圆的东西，那可真的.....\n制作：天气温云', 'state': 0, 'duration': 92, 'mission_id': 1377412, 'rights': {'bp': 0, 'elec': 0, 'download': 0, 'movie': 0, 'pay': 0, 'hd5': 1, 'no_reprint': 1, 'autoplay': 1, 'ugc_pay': 0, 'is_cooperation': 0, 'ugc_pay_preview': 0, 'no_background': 0, 'arc_pay': 0, 'pay_free_watch': 0}, 'owner': {'mid': 110371677, 'name': '天气温云', 'face': 'https://i0.hdslb.com/bfs/face/1638b504df4bcb91a8626f1c96981e03ff086737.jpg'}, 'stat': {'aid': 442960465, 'view': 805827, 'danmaku': 753, 'reply': 485, 'favorite': 6764, 'coin': 3437, 'share': 5599, 'now_rank': 0, 'his_rank': 0, 'like': 27629, 'dislike': 0, 'vt': 0, 'vv': 805827}, 'dynamic': '', 'cid': 1107570473, 'dimension': {'width': 1920, 'height': 1080, 'rotate': 0}, 's

In [77]:
#迭代获取其他视频的推荐
index = 0
whole_related_df = pd.DataFrame()
for it in relate_list:
    danmu_relate = get_prot_dm(it['cid'])
    #preprocess
    listed_danmu = to_list(danmu_relate)
    df = pd.DataFrame(listed_danmu)
    mask = df['content'].str.len() > 10
    df = df[~mask]
    df['content'] = df['content'].str.replace('[^\w\s]', '')
    df['content'] = df['content'].str.replace('哈', '')
    df['content'] = df['content'].str.replace('嘿', '')
    df['content'] = df['content'].str.replace('啊', '')
    mask = df['content'].str.len() <= 1
    df = df[~mask]
    df.dropna(subset=['content'], inplace=True)
    #end of preprocess
    relate_tmp = df['content'].value_counts()
    relate_tmp = pd.DataFrame({'content': relate_tmp.index, 'count': relate_tmp.values})
    relate_tmp['emo'] = relate_tmp['content'].apply(get_sentiment)
    relate_tmp.to_csv("relate_video_"+str(index)+".csv")
    whole_related_df = pd.concat([df, whole_related_df], axis=0)

    index+=1

whole_related_df['id'] /= 100000000
whole_related_df

,id,progress,content
0,1.298707e+10,40683,离谱
2,1.298803e+10,50727,要来力
3,1.298814e+10,25275,让世界爱上中国造
4,1.298818e+10,27081,笑了
6,1.299011e+10,30611,感动泪目了
...,...,...,...
681,1.310419e+10,36184,够阴间
682,1.310450e+10,12117,腾敬萧
683,1.310450e+10,3836,你个头
685,1.310462e+10,37111,你就是个嘬嘬嘬嘬


In [78]:
# whole_count = whole_related_df['content'].value_counts()
# whole_count = pd.DataFrame({'content': whole_count.index, 'count': whole_count.values})
whole_count = whole_related_df.groupby('content').agg({'id': 'sum', 'content': 'size'}).rename(columns={'content': 'count'}).reset_index()
mask = whole_count['count'].values <= 3
whole_count = whole_count[~mask]
whole_count
whole_count['emo'] = whole_count['content'].apply(get_sentiment)

In [79]:
whole_count.to_csv("main.csv")
print(whole_count)
whole_count

          content            id  count  emo
120    3 4 做个好好先生  5.196653e+10      4    1
147        4K频道精选  5.958601e+10      5    5
149        4k频道精选  4.663501e+10      4    5
162            66  1.044417e+11      8    0
163           666  4.698578e+11     36    3
...           ...           ...    ...  ...
16824        鸡裤同笼  1.175777e+11      9    2
16840          鸣人  6.541639e+10      5    5
16907       黑红也是红  6.511902e+10      5    3
16925        타이쿠라  6.514681e+10      5    3
16929         테콜라  5.210563e+10      4    3

[642 rows x 4 columns]


,content,id,count,emo
120,3 4 做个好好先生,5.196653e+10,4,1
147,4K频道精选,5.958601e+10,5,5
149,4k频道精选,4.663501e+10,4,5
162,66,1.044417e+11,8,0
163,666,4.698578e+11,36,3
...,...,...,...,...
16824,鸡裤同笼,1.175777e+11,9,2
16840,鸣人,6.541639e+10,5,5
16907,黑红也是红,6.511902e+10,5,3
16925,타이쿠라,6.514681e+10,5,3


In [84]:
# 降维 用于主图
# 使用t-SNE进行降维
tsne = TSNE(n_components=2, random_state=42)
embedding = tsne.fit_transform(whole_count.drop('content', axis=1))

# 创建降维后的DataFrame
df_tsne = pd.DataFrame(embedding, columns=['Dimension 1', 'Dimension 2'])
df_tsne['content'] = df['content']
df_tsne.to_csv("tsne.csv")

In [ ]:
# # 3.保存弹幕列表
#
# def save_to_file(data):
#     with open("dan_mu.txt", mode="w", encoding="utf-8") as f:
#         for i in data:
#             f.write(i)
#             f.write("\n")
# save_to_file(data)


In [ ]:
# 1 导入相关库
import pandas as pd
import jieba
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from imageio import imread

In [ ]:
# 目标视频文件的处理
df = pd.read_csv("video.csv")
array = np.array(df)
content = array[:, 3]
progress = array

In [ ]:

import warnings
warnings.filterwarnings("ignore")

# 2 读取文本文件，并使用lcut()方法进行分词
with open("video.csv",encoding="utf-8") as f:
    txt = f.read()
txt = txt.split()
data_cut = [jieba.lcut(x) for x in txt]
data_cut
# 3 读取停用词
stop = [" ","道","说道","说",'了']
# 4 去掉停用词之后的最终词
s_data_cut = pd.Series(data_cut)
all_words_after = s_data_cut.apply(lambda x:[i for i in x if i not in stop])
# 5 词频统计
all_words = []
for i in all_words_after:
    all_words.extend(i)
word_count = pd.Series(all_words).value_counts()
# 6 词云图的绘制
# 1）读取背景图片


# 2）设置词云参数
wc = WordCloud(font_path="C:/Users/Windows/fonts/simhei.ttf",
               background_color="white",
               max_words=2000,
               max_font_size=200,
               random_state=42
              )
wc2 = wc.fit_words(word_count)

# 3）绘制词云图
plt.figure(figsize=(16,8), dpi=300)
plt.imshow(wc2)
plt.axis("off")
plt.show()
wc.to_file("ciyun.jpg")

